# Pinecone

##### Load Dependencies

In [ ]:
import os
from openai import OpenAI
from typing_extensions import override
from openai import AssistantEventHandler
from pathlib import Path
from playwright.sync_api import sync_playwright
from agents import Agent, Runner
import tiktoken
import base64
import requests
from sklearn.model_selection import train_test_split
from static import Requests, Roles, Languages
from booger import Error, ErrorDialog, ChatWindow, FileDialog, FileBrowser
from importlib import reload
from pinecone import Pinecone
from pinecone_plugins.assistant.models.chat import Message

##### Initialize Client

In [ ]:
# Initialize a Pinecone client with your API key
pc_key = os.getenv( 'PINECONE_API_KEY' )
pc = Pinecone(  api_key=pc_key )

#### Upsert Data

##### Embed & Upsert

In [ ]:
index_name = 'quickstart'

pc.create_index( name=index_name,
    dimension=1024, # Replace with your small_model dimensions
    metric='cosine', # Replace with your small_model metric
    spec=ServerlessSpec(
        cloud='aws',
        region='us-east-1'
    )
)

##### Query

In [ ]:
query = 'Tell me about the tech company known as Apple'

x = pc.inference.create_vector(
    model='multilingual-e5-large',
    inputs=[query],
    parameters={
        'input_type': 'query'
    }
)

results = index.query(
    namespace='ns1',
    vector=x[0].values,
    top_k=3,
    include_values=False,
    include_metadata=True
)

print(results)

##### Create Vector Embeddings

In [ ]:
##### Create Vector Embeddings
data = [
	{ 'id': 'vec1', 'pages': 'Apple is a popular fruit known for its sweetness and crisp texture.' },
	{ 'id': 'vec2',
	  'pages': 'The tech company Apple is known for its innovative products like the iPhone.' },
	{ 'id': 'vec3', 'pages': 'Many people enjoy eating apples as a healthy snack.' },
	{ 'id': 'vec4',
	  'pages': 'Apple Inc. has revolutionized the tech industry with its sleek designs and user-friendly interfaces.' },
	{ 'id': 'vec5', 'pages': 'An apple a day keeps the doctor away, as the saying goes.' },
	{ 'id': 'vec6',
	  'pages': 'Apple Computer Company was founded on April 1, 1976, by Steve Jobs, Steve Wozniak, and Ronald Wayne as a partnership.' }
]

embeddings = pc.inference.create_vector(
	model='multilingual-e5-large',
	inputs=[ d[ 'pages' ] for d in data ],
	parameters={ 'input_type': 'passage', 'trunc': 'END' }
)
print( embeddings[ 0 ] )

##### Upsert Data

In [ ]:
# Wait for the index to be ready
while not pc.describe_index(index_name).status['ready']:
    time.sleep(1)

index = pc.Index(index_name)

vectors = []
for d, e in zip(data, embeddings):
    vectors.append({
        'id': d['id'],
        'target_values': e['target_values'],
        'metadata': {'pages': d['pages']}
    })

index.upsert(
    vectors=vectors,
    namespace='ns1'
)

##### Check Index

In [ ]:
print(index.describe_index_stats())

##### Upload Files

In [ ]:
pc = Pinecone( api_key=pc_key )
assistant = pc.assistant.Assistant( assistant_name="bubba", )
response = assistant.upload_file( file_path='/Users/jdoe/Downloads/example_file.txt', timeout=None )

##### Chat with Pinecone

In [ ]:
pc = Pinecone (api_key=pc_key )
assistant = pc.assistant.Assistant( assistant_name='bubba' )
msg = Message( content='How old is the earth?')
resp = assistant.chat( messages=[ msg ] )
print( resp[ '"message" '][ 'content' ] )

# With streaming
chunks = assistant.chat( messages=[ msg ], stream=True )
for chunk in chunks:
    if chunk:
        print(chunk)